### Data transformation task
- Retrieve hourly battery profit data from multiple local folders containing .csv files of the battery revenue model output. Transform this data to calculate the monthly profit totals categorized by geography, duration, year, month, and services, ensuring that the final output is formatted for easy pivot-table analysis in Excel for team collaboration.

In [2]:
import os
import glob
import pandas as pd

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [132]:
# AUS

In [3]:
def get_csv_files(folder_path):
    # Construct the search pattern
    search_pattern = os.path.join(folder_path, 'Output*.csv')
    
    # Use glob to find all matching files
    csv_files = glob.glob(search_pattern)
    
    return csv_files

def convert_csvs_to_dataframes(csv_files):
    dataframes = {}
    
    for file in csv_files:
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file)
        
        # Use the file name (without path and extension) as the key
        file_name = os.path.basename(file).replace('.csv', '')
        dataframes[file_name] = df
    
    return dataframes

folder_path = r'C:\Users\tom.kim\OneDrive - S&P Global\Desktop\Data Science\Battery Revenues\results\Australia_All_Services' 
csv_files = get_csv_files(folder_path)

# Convert CSV files to DataFrames
AUS = convert_csvs_to_dataframes(csv_files)


#for name, df in AUS.items():
#    print(f"AUS df for {name}:")
#    print(df.head(), "\n")

In [4]:
# Function to filter DataFrames
def filter_columns(df):
    return df.filter(regex=r'(^EnergyProfit|Rev_AUD$)')

In [5]:
filtered_AUS = {name: filter_columns(df) for name, df in AUS.items()}

In [6]:
dates_2022 = pd.date_range(start='2022/1/1 00:00:00', end='2022/12/31 23:00:00', freq="h")
dates_2023 = pd.date_range(start='2023/1/1 00:00:00', end='2023/12/31 23:00:00', freq="h")
dates_2024 = pd.date_range(start='2024/1/1 00:00:00', end='2024/12/30 23:00:00', freq="h")

In [7]:
filtered_AUS['Outputs_2022_0.5_2025_02_05'].set_index(dates_2022, inplace=True)
filtered_AUS['Outputs_2022_1_2025_02_05'].set_index(dates_2022, inplace=True)
filtered_AUS['Outputs_2022_2_2025_02_05'].set_index(dates_2022, inplace=True)
filtered_AUS['Outputs_2022_4_2025_02_05'].set_index(dates_2022, inplace=True)
filtered_AUS['Outputs_2023_0.5_2025_02_05'].set_index(dates_2023, inplace=True)
filtered_AUS['Outputs_2023_1_2025_02_05'].set_index(dates_2023, inplace=True)
filtered_AUS['Outputs_2023_2_2025_02_05'].set_index(dates_2023, inplace=True)
filtered_AUS['Outputs_2023_4_2025_02_05'].set_index(dates_2023, inplace=True)
filtered_AUS['Outputs_2024_0.5_2025_02_05'].set_index(dates_2024, inplace=True)
filtered_AUS['Outputs_2024_1_2025_02_05'].set_index(dates_2024, inplace=True)
filtered_AUS['Outputs_2024_2_2025_02_05'].set_index(dates_2024, inplace=True)
filtered_AUS['Outputs_2024_4_2025_02_05'].set_index(dates_2024, inplace=True)

In [8]:
# Resample each DataFrame to monthly sums and add new columns
for name, df in filtered_AUS.items():
    # Resample to monthly sums
    df_resampled = df.resample('MS').sum()
    
    # Extract year and duration from the DataFrame name
    geography = name.split('_')[0]
    year = name.split('_')[1]
    duration = name.split('_')[2]
    
    # Add new columns
    df_resampled['geography'] = 'AUS'
    df_resampled['year'] = year
    df_resampled['duration'] = duration
    df_resampled['month'] = df_resampled.index.month
    
    # Update the dictionary with the modified DataFrame
    filtered_AUS[name] = df_resampled

# Concatenate all DataFrames into a single DataFrame
AUS_combined = pd.concat(filtered_AUS.values(), axis=0)

# Reset index if needed
AUS_combined.reset_index(drop=True, inplace=True)

In [9]:
AUS_combined

,EnergyProfit_AUD,L1SRev_AUD,R6SRev_AUD,R1SRev_AUD,R60SRev_AUD,L60SRev_AUD,RREGRev_AUD,LREGRev_AUD,R5MRev_AUD,L6SRev_AUD,geography,year,duration,month
0,3151.17,0.00,507.11,0.00,431.58,885.50,4659.98,3343.88,12.11,11.52,AUS,2022,0.5,1
1,964.79,0.00,361.87,0.00,308.42,1176.22,4138.73,3600.66,13.10,35.60,AUS,2022,0.5,2
2,1398.56,0.00,810.87,0.00,529.36,585.17,5244.90,3657.33,15.70,39.43,AUS,2022,0.5,3
3,3238.40,0.00,2380.95,0.00,137.62,225.59,7254.62,3733.34,7.19,89.74,AUS,2022,0.5,4
4,3080.94,0.00,4254.20,0.00,469.75,306.71,10209.97,4990.95,1.58,76.02,AUS,2022,0.5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,62539.37,1.71,19.35,912.55,4.19,242.37,1525.17,984.06,12.53,18.72,AUS,2024,4,8
140,13166.67,8.45,26.18,1884.06,1.95,208.17,1152.38,904.42,12.41,46.15,AUS,2024,4,9
141,20436.89,29.16,7.90,1564.73,1.33,71.04,1683.44,1133.71,8.33,24.89,AUS,2024,4,10
142,18710.52,3.22,16.57,1000.23,0.86,49.91,2356.58,960.73,9.49,28.69,AUS,2024,4,11


In [10]:
AUS_melted = pd.melt(AUS_combined, id_vars=['geography', 'year', 'duration', 'month'], var_name='revenue_currency', value_name='profit')

In [11]:
AUS_melted

,geography,year,duration,month,revenue_currency,profit
0,AUS,2022,0.5,1,EnergyProfit_AUD,3151.17
1,AUS,2022,0.5,2,EnergyProfit_AUD,964.79
2,AUS,2022,0.5,3,EnergyProfit_AUD,1398.56
3,AUS,2022,0.5,4,EnergyProfit_AUD,3238.40
4,AUS,2022,0.5,5,EnergyProfit_AUD,3080.94
...,...,...,...,...,...,...
1435,AUS,2024,4,8,L6SRev_AUD,18.72
1436,AUS,2024,4,9,L6SRev_AUD,46.15
1437,AUS,2024,4,10,L6SRev_AUD,24.89
1438,AUS,2024,4,11,L6SRev_AUD,28.69


In [12]:
# CAISO

In [13]:
folder_path = r'C:\Users\tom.kim\OneDrive - S&P Global\Desktop\Data Science\Battery Revenues\results\CAISO' 
csv_files = get_csv_files(folder_path)

# Convert CSV files to DataFrames
CAISO = convert_csvs_to_dataframes(csv_files)

In [14]:
# Function to filter DataFrames
def filter_columns(df):
    return df.filter(regex=r'(^EnergyProfit|Rev)')

In [15]:
filtered_CAISO = {name: filter_columns(df) for name, df in CAISO.items()}

In [16]:

#filtered_CAISO

In [17]:
dates_2021 = pd.date_range(start='2021/1/1 00:00:00', end='2021/12/31 23:00:00', freq="h")
dates_2022 = pd.date_range(start='2022/1/1 00:00:00', end='2022/12/31 23:00:00', freq="h")
dates_2023 = pd.date_range(start='2023/1/1 00:00:00', end='2023/12/31 23:00:00', freq="h")
dates_2024 = pd.date_range(start='2024/1/1 00:00:00', end='2024/12/31 23:00:00', freq="h")

In [18]:
filtered_CAISO['Outputs_2021_0.5_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_CAISO['Outputs_2021_1_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_CAISO['Outputs_2021_2_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_CAISO['Outputs_2021_4_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_CAISO['Outputs_2022_0.5_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_CAISO['Outputs_2022_1_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_CAISO['Outputs_2022_2_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_CAISO['Outputs_2022_4_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_CAISO['Outputs_2023_0.5_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_CAISO['Outputs_2023_1_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_CAISO['Outputs_2023_2_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_CAISO['Outputs_2023_4_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_CAISO['Outputs_2024_0.5_2025_01_17'].set_index(dates_2024, inplace=True)
filtered_CAISO['Outputs_2024_1_2025_01_17'].set_index(dates_2024, inplace=True)
filtered_CAISO['Outputs_2024_2_2025_01_17'].set_index(dates_2024, inplace=True)
filtered_CAISO['Outputs_2024_4_2025_01_17'].set_index(dates_2024, inplace=True)

In [19]:
filtered_CAISO['Outputs_2024_0.5_2025_01_17'].rename(columns=lambda x: x.replace('$', 'USD'), inplace=True)
filtered_CAISO['Outputs_2024_1_2025_01_17'].rename(columns=lambda x: x.replace('$', 'USD'), inplace=True)
filtered_CAISO['Outputs_2024_2_2025_01_17'].rename(columns=lambda x: x.replace('$', 'USD'), inplace=True)
filtered_CAISO['Outputs_2024_4_2025_01_17'].rename(columns=lambda x: x.replace('$', 'USD'), inplace=True)

In [20]:
# Resample each DataFrame to monthly sums and add new columns
for name, df in filtered_CAISO.items():
    # Resample to monthly sums
    df_resampled = df.resample('MS').sum()
    
    # Extract year and duration from the DataFrame name
    geography = name.split('_')[0]
    year = name.split('_')[1]
    duration = name.split('_')[2]
    
    # Add new columns
    df_resampled['geography'] = 'CAISO'
    df_resampled['year'] = year
    df_resampled['duration'] = duration
    df_resampled['month'] = df_resampled.index.month
    
    # Update the dictionary with the modified DataFrame
    filtered_CAISO[name] = df_resampled

# Concatenate all DataFrames into a single DataFrame
CAISO_combined = pd.concat(filtered_CAISO.values(), axis=0)

# Reset index if needed
CAISO_combined.reset_index(drop=True, inplace=True)

In [21]:
CAISO_combined

,EnergyProfit_USD,non_spinRev_USD,spinRev_USD,reg_downRev_USD,reg_upRev_USD,geography,year,duration,month
0,180.29,0.02,631.33,1760.43,847.35,CAISO,2021,0.5,1
1,1913.89,0.00,1327.05,2716.75,1299.12,CAISO,2021,0.5,2
2,564.49,0.10,298.09,2053.81,715.40,CAISO,2021,0.5,3
3,582.02,0.00,303.45,1950.93,695.86,CAISO,2021,0.5,4
4,567.89,0.01,433.51,1930.20,566.95,CAISO,2021,0.5,5
...,...,...,...,...,...,...,...,...,...
187,3987.35,578.84,279.25,207.78,99.62,CAISO,2024,4,8
188,5137.81,275.62,345.90,203.02,78.30,CAISO,2024,4,9
189,3676.54,347.04,290.63,180.24,70.38,CAISO,2024,4,10
190,4074.50,42.41,74.07,197.02,47.84,CAISO,2024,4,11


In [22]:
CAISO_melted = pd.melt(CAISO_combined, id_vars=['geography', 'year', 'duration', 'month'], var_name='revenue_currency', value_name='profit')

In [23]:
CAISO_melted

,geography,year,duration,month,revenue_currency,profit
0,CAISO,2021,0.5,1,EnergyProfit_USD,180.29
1,CAISO,2021,0.5,2,EnergyProfit_USD,1913.89
2,CAISO,2021,0.5,3,EnergyProfit_USD,564.49
3,CAISO,2021,0.5,4,EnergyProfit_USD,582.02
4,CAISO,2021,0.5,5,EnergyProfit_USD,567.89
...,...,...,...,...,...,...
955,CAISO,2024,4,8,reg_upRev_USD,99.62
956,CAISO,2024,4,9,reg_upRev_USD,78.30
957,CAISO,2024,4,10,reg_upRev_USD,70.38
958,CAISO,2024,4,11,reg_upRev_USD,47.84


In [24]:
# China

In [25]:
folder_path = r'C:\Users\tom.kim\OneDrive - S&P Global\Desktop\Data Science\Battery Revenues\results\China' 
csv_files = get_csv_files(folder_path)

# Convert CSV files to DataFrames
China = convert_csvs_to_dataframes(csv_files)

In [26]:
filtered_China = {name: filter_columns(df) for name, df in China.items()}

In [27]:
dates_2022 = pd.date_range(start='2022/1/1 00:00:00', end='2022/12/31 23:00:00', freq="h")
dates_2023 = pd.date_range(start='2023/1/1 00:00:00', end='2023/12/31 23:00:00', freq="h")
dates_2024 = pd.date_range(start='2024/1/1 00:00:00', end='2024/12/30 23:00:00', freq="h")

In [28]:
filtered_China['Outputs_2022_0.5_2025_02_06'].set_index(dates_2022, inplace=True)
filtered_China['Outputs_2022_1_2025_02_06'].set_index(dates_2022, inplace=True)
filtered_China['Outputs_2022_2_2025_02_06'].set_index(dates_2022, inplace=True)
filtered_China['Outputs_2022_4_2025_02_06'].set_index(dates_2022, inplace=True)
filtered_China['Outputs_2023_0.5_2025_02_06'].set_index(dates_2023, inplace=True)
filtered_China['Outputs_2023_1_2025_02_06'].set_index(dates_2023, inplace=True)
filtered_China['Outputs_2023_2_2025_02_06'].set_index(dates_2023, inplace=True)
filtered_China['Outputs_2023_4_2025_02_06'].set_index(dates_2023, inplace=True)
filtered_China['Outputs_2024_0.5_2025_02_06'].set_index(dates_2024, inplace=True)
filtered_China['Outputs_2024_1_2025_02_06'].set_index(dates_2024, inplace=True)
filtered_China['Outputs_2024_2_2025_02_06'].set_index(dates_2024, inplace=True)
filtered_China['Outputs_2024_4_2025_02_06'].set_index(dates_2024, inplace=True)

In [29]:
# Resample each DataFrame to monthly sums and add new columns
for name, df in filtered_China.items():
    # Resample to monthly sums
    df_resampled = df.resample('MS').sum()
    
    # Extract year and duration from the DataFrame name
    geography = name.split('_')[0]
    year = name.split('_')[1]
    duration = name.split('_')[2]
    
    # Add new columns
    df_resampled['geography'] = 'China'
    df_resampled['year'] = year
    df_resampled['duration'] = duration
    df_resampled['month'] = df_resampled.index.month
    
    # Update the dictionary with the modified DataFrame
    filtered_China[name] = df_resampled

# Concatenate all DataFrames into a single DataFrame
China_combined = pd.concat(filtered_China.values(), axis=0)

# Reset index if needed
China_combined.reset_index(drop=True, inplace=True)

In [30]:
China_melted = pd.melt(China_combined, id_vars=['geography', 'year', 'duration', 'month'], var_name='revenue_currency', value_name='profit')

In [31]:
China_melted

,geography,year,duration,month,revenue_currency,profit
0,China,2022,0.5,1,EnergyProfit_¥,10535.58
1,China,2022,0.5,2,EnergyProfit_¥,11712.37
2,China,2022,0.5,3,EnergyProfit_¥,8695.84
3,China,2022,0.5,4,EnergyProfit_¥,8116.57
4,China,2022,0.5,5,EnergyProfit_¥,8211.89
...,...,...,...,...,...,...
139,China,2024,4,8,EnergyProfit_¥,24537.14
140,China,2024,4,9,EnergyProfit_¥,28031.36
141,China,2024,4,10,EnergyProfit_¥,43801.89
142,China,2024,4,11,EnergyProfit_¥,40840.86


In [32]:
# UK

In [33]:
folder_path = r'C:\Users\tom.kim\OneDrive - S&P Global\Desktop\Data Science\Battery Revenues\results\UK' 
csv_files = get_csv_files(folder_path)

# Convert CSV files to DataFrames
UK = convert_csvs_to_dataframes(csv_files)

In [34]:
# Function to filter DataFrames
def filter_columns(df):
    return df.filter(regex=r'(^EnergyProfit|Rev_USD)')

In [35]:
filtered_UK = {name: filter_columns(df) for name, df in UK.items()}

In [36]:
dates_2021 = pd.date_range(start='2021/1/1 00:00:00', end='2021/12/31 23:00:00', freq="h")
dates_2022 = pd.date_range(start='2022/1/1 00:00:00', end='2022/12/31 23:00:00', freq="h")
dates_2023 = pd.date_range(start='2023/1/1 00:00:00', end='2023/12/31 23:00:00', freq="h")
dates_2024 = pd.date_range(start='2024/1/1 00:00:00', end='2024/12/31 23:00:00', freq="h")

In [37]:
filtered_UK['Outputs_2021_0.5_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_UK['Outputs_2021_1_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_UK['Outputs_2021_2_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_UK['Outputs_2021_4_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_UK['Outputs_2022_0.5_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_UK['Outputs_2022_1_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_UK['Outputs_2022_2_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_UK['Outputs_2022_4_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_UK['Outputs_2023_0.5_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_UK['Outputs_2023_1_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_UK['Outputs_2023_2_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_UK['Outputs_2023_4_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_UK['Outputs_2024_0.5_2025_01_15'].set_index(dates_2024, inplace=True)
filtered_UK['Outputs_2024_1_2025_01_15'].set_index(dates_2024, inplace=True)
filtered_UK['Outputs_2024_2_2025_01_15'].set_index(dates_2024, inplace=True)
filtered_UK['Outputs_2024_4_2025_01_15'].set_index(dates_2024, inplace=True)

In [38]:
# Resample each DataFrame to monthly sums and add new columns
for name, df in filtered_UK.items():
    # Resample to monthly sums
    df_resampled = df.resample('MS').sum()
    
    # Extract year and duration from the DataFrame name
    geography = name.split('_')[0]
    year = name.split('_')[1]
    duration = name.split('_')[2]
    
    # Add new columns
    df_resampled['geography'] = 'UK'
    df_resampled['year'] = year
    df_resampled['duration'] = duration
    df_resampled['month'] = df_resampled.index.month
    
    # Update the dictionary with the modified DataFrame
    filtered_UK[name] = df_resampled

# Concatenate all DataFrames into a single DataFrame
UK_combined = pd.concat(filtered_UK.values(), axis=0)

# Reset index if needed
UK_combined.reset_index(drop=True, inplace=True)

In [39]:
UK_melted = pd.melt(UK_combined, id_vars=['geography', 'year', 'duration', 'month'], var_name='revenue_currency', value_name='profit')

In [40]:
UK_melted

,geography,year,duration,month,revenue_currency,profit
0,UK,2021,0.5,1,EnergyProfit_USD,1551.65
1,UK,2021,0.5,2,EnergyProfit_USD,-1278.11
2,UK,2021,0.5,3,EnergyProfit_USD,338.29
3,UK,2021,0.5,4,EnergyProfit_USD,-1640.02
4,UK,2021,0.5,5,EnergyProfit_USD,-1990.86
...,...,...,...,...,...,...
2299,UK,2024,4,8,PQRRev_USD,0.00
2300,UK,2024,4,9,PQRRev_USD,0.00
2301,UK,2024,4,10,PQRRev_USD,0.00
2302,UK,2024,4,11,PQRRev_USD,0.00


In [41]:
# ERCOT

In [42]:
folder_path = r'C:\Users\tom.kim\OneDrive - S&P Global\Desktop\Data Science\Battery Revenues\results\ERCOT_All_Services_with_Uri' 
csv_files = get_csv_files(folder_path)

# Convert CSV files to DataFrames
ERCOT = convert_csvs_to_dataframes(csv_files)

In [43]:
# Function to filter DataFrames
def filter_columns(df):
    return df.filter(regex=r'(^EnergyProfit|Rev)')

In [44]:
filtered_ERCOT = {name: filter_columns(df) for name, df in ERCOT.items()}

In [45]:
#filtered_ERCOT

In [46]:
dates_2021 = pd.date_range(start='2021/1/1 00:00:00', end='2021/12/31 23:00:00', freq="h")
dates_2022 = pd.date_range(start='2022/1/1 00:00:00', end='2022/12/31 23:00:00', freq="h")
dates_2023 = pd.date_range(start='2023/1/1 00:00:00', end='2023/12/31 23:00:00', freq="h")
dates_2024 = pd.date_range(start='2024/1/1 00:00:00', end='2024/12/31 23:00:00', freq="h")

In [47]:
filtered_ERCOT['Outputs_2021_0.5_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_ERCOT['Outputs_2021_1_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_ERCOT['Outputs_2021_2_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_ERCOT['Outputs_2021_4_2024_11_15'].set_index(dates_2021, inplace=True)
filtered_ERCOT['Outputs_2022_0.5_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_ERCOT['Outputs_2022_1_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_ERCOT['Outputs_2022_2_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_ERCOT['Outputs_2022_4_2024_11_15'].set_index(dates_2022, inplace=True)
filtered_ERCOT['Outputs_2023_0.5_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_ERCOT['Outputs_2023_1_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_ERCOT['Outputs_2023_2_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_ERCOT['Outputs_2023_4_2024_11_15'].set_index(dates_2023, inplace=True)
filtered_ERCOT['Outputs_2024_0.5_2025_02_06'].set_index(dates_2024, inplace=True)
filtered_ERCOT['Outputs_2024_1_2025_02_06'].set_index(dates_2024, inplace=True)
filtered_ERCOT['Outputs_2024_2_2025_02_06'].set_index(dates_2024, inplace=True)
filtered_ERCOT['Outputs_2024_4_2025_02_06'].set_index(dates_2024, inplace=True)

In [48]:
filtered_ERCOT['Outputs_2024_0.5_2025_02_06'].rename(columns=lambda x: x.replace('$', 'USD'), inplace=True)
filtered_ERCOT['Outputs_2024_1_2025_02_06'].rename(columns=lambda x: x.replace('$', 'USD'), inplace=True)
filtered_ERCOT['Outputs_2024_2_2025_02_06'].rename(columns=lambda x: x.replace('$', 'USD'), inplace=True)
filtered_ERCOT['Outputs_2024_4_2025_02_06'].rename(columns=lambda x: x.replace('$', 'USD'), inplace=True)

In [49]:
# Resample each DataFrame to monthly sums and add new columns
for name, df in filtered_ERCOT.items():
    # Resample to monthly sums
    df_resampled = df.resample('MS').sum()
    
    # Extract year and duration from the DataFrame name
    geography = name.split('_')[0]
    year = name.split('_')[1]
    duration = name.split('_')[2]
    
    # Add new columns
    df_resampled['geography'] = 'ERCOT'
    df_resampled['year'] = year
    df_resampled['duration'] = duration
    df_resampled['month'] = df_resampled.index.month
    
    # Update the dictionary with the modified DataFrame
    filtered_ERCOT[name] = df_resampled

# Concatenate all DataFrames into a single DataFrame
ERCOT_combined = pd.concat(filtered_ERCOT.values(), axis=0)

# Reset index if needed
ERCOT_combined.reset_index(drop=True, inplace=True)

In [50]:
ERCOT_melted = pd.melt(ERCOT_combined, id_vars=['geography', 'year', 'duration', 'month'], var_name='revenue_currency', value_name='profit')

In [51]:
ERCOT_melted

,geography,year,duration,month,revenue_currency,profit
0,ERCOT,2021,0.5,1,EnergyProfit_USD,-13.83
1,ERCOT,2021,0.5,2,EnergyProfit_USD,567.34
2,ERCOT,2021,0.5,3,EnergyProfit_USD,92.79
3,ERCOT,2021,0.5,4,EnergyProfit_USD,288.20
4,ERCOT,2021,0.5,5,EnergyProfit_USD,22.38
...,...,...,...,...,...,...
1147,ERCOT,2024,4,8,rrsRev_USD,1532.23
1148,ERCOT,2024,4,9,rrsRev_USD,288.64
1149,ERCOT,2024,4,10,rrsRev_USD,646.80
1150,ERCOT,2024,4,11,rrsRev_USD,263.04


In [52]:
# Germany

In [53]:
folder_path = r'C:\Users\tom.kim\OneDrive - S&P Global\Desktop\Data Science\Battery Revenues\results\Germany_2022-24' 
csv_files = get_csv_files(folder_path)

# Convert CSV files to DataFrames
Germany = convert_csvs_to_dataframes(csv_files)

In [54]:
# Function to filter DataFrames
def filter_columns(df):
    return df.filter(regex=r'(^EnergyProfit|Rev)')

In [55]:
filtered_Germany = {name: filter_columns(df) for name, df in Germany.items()}

In [56]:
dates_2022 = pd.date_range(start='2022/1/1 00:00:00', end='2022/12/31 23:45:00', freq="15T")
dates_2023 = pd.date_range(start='2023/1/1 00:00:00', end='2023/12/31 23:45:00', freq="15T")
dates_2024 = pd.date_range(start='2024/1/1 00:00:00', end='2024/12/30 23:45:00', freq="15T")

In [57]:
#dates_2022

In [58]:
filtered_Germany['Outputs_2022_0.5_2025_02_12'].set_index(dates_2022, inplace=True)
filtered_Germany['Outputs_2022_1_2025_02_12'].set_index(dates_2022, inplace=True)
filtered_Germany['Outputs_2022_2_2025_02_12'].set_index(dates_2022, inplace=True)
filtered_Germany['Outputs_2022_4_2025_02_12'].set_index(dates_2022, inplace=True)
filtered_Germany['Outputs_2023_0.5_2025_02_12'].set_index(dates_2023, inplace=True)
filtered_Germany['Outputs_2023_1_2025_02_12'].set_index(dates_2023, inplace=True)
filtered_Germany['Outputs_2023_2_2025_02_12'].set_index(dates_2023, inplace=True)
filtered_Germany['Outputs_2023_4_2025_02_12'].set_index(dates_2023, inplace=True)
filtered_Germany['Outputs_2024_0.5_2025_02_12'].set_index(dates_2024, inplace=True)
filtered_Germany['Outputs_2024_1_2025_02_12'].set_index(dates_2024, inplace=True)
filtered_Germany['Outputs_2024_2_2025_02_12'].set_index(dates_2024, inplace=True)
filtered_Germany['Outputs_2024_4_2025_02_12'].set_index(dates_2024, inplace=True)

In [59]:
# Resample each DataFrame to monthly sums and add new columns
for name, df in filtered_Germany.items():
    # Resample to monthly sums
    df_resampled = df.resample('MS').sum()
    
    # Extract year and duration from the DataFrame name
    geography = name.split('_')[0]
    year = name.split('_')[1]
    duration = name.split('_')[2]
    
    # Add new columns
    df_resampled['geography'] = 'Germany'
    df_resampled['year'] = year
    df_resampled['duration'] = duration
    df_resampled['month'] = df_resampled.index.month
    
    # Update the dictionary with the modified DataFrame
    filtered_Germany[name] = df_resampled

# Concatenate all DataFrames into a single DataFrame
Germany_combined = pd.concat(filtered_Germany.values(), axis=0)

# Reset index if needed
Germany_combined.reset_index(drop=True, inplace=True)

In [60]:
Germany_melted = pd.melt(Germany_combined, id_vars=['geography', 'year', 'duration', 'month'], var_name='revenue_currency', value_name='profit')

In [61]:
Germany_melted

,geography,year,duration,month,revenue_currency,profit
0,Germany,2022,0.5,1,EnergyProfit_€,4987.69
1,Germany,2022,0.5,2,EnergyProfit_€,937.96
2,Germany,2022,0.5,3,EnergyProfit_€,3583.22
3,Germany,2022,0.5,4,EnergyProfit_€,1353.72
4,Germany,2022,0.5,5,EnergyProfit_€,216.97
...,...,...,...,...,...,...
571,Germany,2024,4,8,aFRR_DownRev_€,5474.60
572,Germany,2024,4,9,aFRR_DownRev_€,5319.47
573,Germany,2024,4,10,aFRR_DownRev_€,3215.28
574,Germany,2024,4,11,aFRR_DownRev_€,1457.51


In [ ]:
# Japan

In [62]:
folder_path = r'C:\Users\tom.kim\OneDrive - S&P Global\Desktop\Data Science\Battery Revenues\results\Japan_All_Services' 
csv_files = get_csv_files(folder_path)

# Convert CSV files to DataFrames
Japan = convert_csvs_to_dataframes(csv_files)

In [63]:
# Function to filter DataFrames
def filter_columns(df):
    return df.filter(regex=r'(^EnergyProfit|Rev)')

In [64]:
filtered_Japan = {name: filter_columns(df) for name, df in Japan.items()}

In [67]:
dates_2022 = pd.date_range(start='2022/1/1 00:00:00', end='2022/12/31 23:45:00', freq="30t")
dates_2023 = pd.date_range(start='2023/1/1 00:00:00', end='2023/12/31 23:45:00', freq="30t")
dates_2024 = pd.date_range(start='2024/1/1 00:00:00', end='2024/12/30 23:45:00', freq="30t")

In [68]:
filtered_Japan['Outputs_2022_1_2025_02_17'].set_index(dates_2022, inplace=True)
filtered_Japan['Outputs_2022_2_2025_02_17'].set_index(dates_2022, inplace=True)
filtered_Japan['Outputs_2022_3_2025_02_17'].set_index(dates_2022, inplace=True)
filtered_Japan['Outputs_2022_4_2025_02_17'].set_index(dates_2022, inplace=True)
filtered_Japan['Outputs_2023_1_2025_02_17'].set_index(dates_2023, inplace=True)
filtered_Japan['Outputs_2023_2_2025_02_17'].set_index(dates_2023, inplace=True)
filtered_Japan['Outputs_2023_3_2025_02_17'].set_index(dates_2023, inplace=True)
filtered_Japan['Outputs_2023_4_2025_02_17'].set_index(dates_2023, inplace=True)
filtered_Japan['Outputs_2024_1_2025_02_17'].set_index(dates_2024, inplace=True)
filtered_Japan['Outputs_2024_2_2025_02_17'].set_index(dates_2024, inplace=True)
filtered_Japan['Outputs_2024_3_2025_02_17'].set_index(dates_2024, inplace=True)
filtered_Japan['Outputs_2024_4_2025_02_17'].set_index(dates_2024, inplace=True)

In [72]:
# Resample each DataFrame to monthly sums and add new columns
for name, df in filtered_Japan.items():
    # Resample to monthly sums
    df_resampled = df.resample('MS').sum()
    
    # Extract year and duration from the DataFrame name
    geography = name.split('_')[0]
    year = name.split('_')[1]
    duration = name.split('_')[2]
    
    # Add new columns
    df_resampled['geography'] = 'Japan'
    df_resampled['year'] = year
    df_resampled['duration'] = duration
    df_resampled['month'] = df_resampled.index.month
    
    # Update the dictionary with the modified DataFrame
    filtered_Japan[name] = df_resampled

# Concatenate all DataFrames into a single DataFrame
Japan_combined = pd.concat(filtered_Japan.values(), axis=0)

# Reset index if needed
Japan_combined.reset_index(drop=True, inplace=True)

In [73]:
Japan_melted = pd.melt(Japan_combined, id_vars=['geography', 'year', 'duration', 'month'], var_name='revenue_currency', value_name='profit')

In [74]:
Japan_melted

,geography,year,duration,month,revenue_currency,profit
0,Japan,2022,1,1,EnergyProfit_JPY,1127210.14
1,Japan,2022,1,2,EnergyProfit_JPY,657854.56
2,Japan,2022,1,3,EnergyProfit_JPY,942146.04
3,Japan,2022,1,4,EnergyProfit_JPY,337018.17
4,Japan,2022,1,5,EnergyProfit_JPY,239937.85
...,...,...,...,...,...,...
859,Japan,2024,4,8,secondary_2Rev_JPY,817771.51
860,Japan,2024,4,9,secondary_2Rev_JPY,740650.43
861,Japan,2024,4,10,secondary_2Rev_JPY,1030229.23
862,Japan,2024,4,11,secondary_2Rev_JPY,847347.97


In [75]:
data_combined = pd.concat([AUS_melted, CAISO_melted, China_melted, UK_melted, ERCOT_melted, Germany_melted, Japan_melted], axis=0, ignore_index=True)

In [76]:
data_combined['category'] = data_combined['revenue_currency'].apply(
    lambda x: 'Energy' if x.startswith('EnergyProfit') else 'Ancillary Service'
)

In [77]:
data_combined = data_combined[['geography', 'year', 'month', 'duration', 'category', 'revenue_currency', 'profit']]

In [78]:
data_combined

,geography,year,month,duration,category,revenue_currency,profit
0,AUS,2022,1,0.5,Energy,EnergyProfit_AUD,3151.17
1,AUS,2022,2,0.5,Energy,EnergyProfit_AUD,964.79
2,AUS,2022,3,0.5,Energy,EnergyProfit_AUD,1398.56
3,AUS,2022,4,0.5,Energy,EnergyProfit_AUD,3238.40
4,AUS,2022,5,0.5,Energy,EnergyProfit_AUD,3080.94
...,...,...,...,...,...,...,...
7435,Japan,2024,8,4,Ancillary Service,secondary_2Rev_JPY,817771.51
7436,Japan,2024,9,4,Ancillary Service,secondary_2Rev_JPY,740650.43
7437,Japan,2024,10,4,Ancillary Service,secondary_2Rev_JPY,1030229.23
7438,Japan,2024,11,4,Ancillary Service,secondary_2Rev_JPY,847347.97


In [79]:
data_combined.to_csv(r'C:\Users\tom.kim\OneDrive - S&P Global\Documents - Energy Storage\Report Production\Reports\Topical\Battery Revenues\result_cleaned_data_Feb192025.csv', index=False)